## 0. Load packages and define auxiliary functions 

In [76]:
# installation of packages
!pip install numpy
!pip install matplotlib

# import of packages 
import numpy as np # used for efficient vector computation
import matplotlib.pyplot as plt # used for plotting
import matplotlib.animation as animation # used for animation of plots
from IPython.display import HTML # used for showing animations 

# auxiliary functions for animation

def init():
    # initializes figure and sets desgin of image
    global fig, ax, im
    fig = plt.figure(1)
    # image with cmap and color bar
    im = plt.imshow((T_tot[:,:,0]-273.15),cmap=plt.get_cmap('hot'), vmin=T_initial-273.15,vmax=T_hot-273.15,extent=[0,w,h,0])
    plt.xlabel('[cm]')
    plt.ylabel('[cm]')
    cbar_ax = fig.add_axes([0.9, 0.15, 0.03, 0.7]) 
    cbar_ax.set_xlabel('$T$ / °C', labelpad=20)
    fig.colorbar(im, cax=cbar_ax)
    # initialize image with temperature at time 0
    im.set_data(T_tot[:,:,0])
    plt.close()
    return 
    
def animate(i):
    # changes image of temperature to time: i*dt 
    global fig, T_tot
    fig.suptitle('{:.1f} min'.format(simulation_time/nframes*i))  # set title above image to current time
    im.set_data(T_tot[:,:,i]-273.15) # change temperature to i-th temperature
    return 

def get_simulation_options(alpha, w, h, simulation_time):
    ### simulation parameter ###
    global dx2,dy2,dt,nsteps,nframes,dnframes,nx,ny,dx,dy
    dx,dy = 0.1,0.1      # finite step in x,y direction [mm]
    dx2, dy2 = dx*dx, dy*dy  #[mm²]
    dt = dx2 * dy2 / (2 * alpha * (dx2 + dy2)) # discretized timestep [ms]
    
    nx, ny = int(w/dx),int(h/dy) # number of nodes in x,y direction 
    nsteps = int(simulation_time*60/dt*1000) # number of timesteps
    nframes = 40 # number of frames
    dnframes = int(nsteps/nframes) # number of timesteps in a single frame
    return 

# auxiliary function to get sample points for comparison with real data
def get_sample(T_tot):
    x_coordinates = [7.5,20.5] # x-coordinates of samples [cm]
    y_coordinates = [2,6] # y-coordinates of samples [cm]
    ts = [4,8,12] # times when samples are taken [min]
   
    simulated_T = [] # initialize simulated_T
    for t in ts:
        for y in y_coordinates:
            for x in x_coordinates:
                simulated_T = np.append(simulated_T,T_tot[int(x/dx),int(y/dy),int(t/dt*60*1000/dnframes)])
    return simulated_T 

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## 1. Load experimental data

Below we load the experimental data. 

In [77]:
# real data, measured 3 times at 4 points 
real_T = np.array([366.77  , 365.18, 299.794 , 298.2,
                   393.516, 393.76, 309.8, 310.19 ,
                   408.638, 407.997, 326.82, 322.4])

## 2. Set the thermal diffusivity $\alpha$

__Hint: Your value should be in the range of $8*10^{-6}$ to $14*10^{-6}$__

In [78]:
alpha = 11.72 * 10**(-6)  # thermal diffusivity [m²/s]

## 3. Finite differences implementation 

The temperature $T_{x_n,y_n}^{n+1}$ at the subsequent time step $t_{n+1}$ is approximated by:

$\Large T_{x_n,y_n}^{t_{n+1}} = T_{x_n,y_n}^{t_n} + \alpha \Delta t (\frac{T_{x_n,y_{n+1}}^{t_n} - 2 T_{x_n,y_{n}}^{t_n}+ T_{x_n,y_{n-1}}^{t_n}}{\Delta y^2} + \frac{T_{x_{n+1},y_{n}}^{t_n} - 2 T_{x_n,y_{n}}^{t_n} + T_{x_{n-1},y_{n}}^{t_n}}{\Delta x^2}) $


In [79]:
# geometrical parameter
w = 26       # width of plate [mm]
h = 8       # height of plate [mm]

# temperatures
T_initial = 24 + 273.15   # initial temperature [K] (24°)
T_hot = 200 + 273.15   # hot temperature on left side [K] (200°C)

simulation_time = 12 # simulation time [min] 



In [80]:
def do_euler_foward(T):
    # compute temperature profile through central difference in space and Euler forward in time
    for i in range(dnframes):
        T[1:-1, 1:-1] = T[1:-1, 1:-1] + alpha * dt * (
              (T[2:, 1:-1] - 2*T[1:-1, 1:-1] + T[:-2, 1:-1])/dx2
              + (T[1:-1, 2:] - 2*T[1:-1, 1:-1] + T[1:-1, :-2])/dy2 )
        # handle outer boundaries  
        T[:,0] = T_hot     # left edge is isothermal at T_hot
        T[:,-1] = T[:,-2]  # right edge is adiabat 
        T[0,:] = T[1,:]    # upper edge is adiabat
        T[-1,:] = T[-2,:]  # lower edge is adiabat     
    return T

# get simulation parameters
get_simulation_options(alpha, w, h, simulation_time)

### Initialization ###
T_tot = np.zeros([nx,ny,nframes+1]) # initialize temperature tensor T_tot(x,y,t)
T_tot[:,:,0] = T_initial * np.ones((nx, ny)) # set initial temperature at t0


### Simulation ###
for i in range(nframes): # iterate through timesteps
    T_tot[:,:,i+1]= do_euler_foward(T_tot[:,:,i].copy()) 
    

# 4. Animation

In [81]:
plt.ion()
init()
ani = animation.FuncAnimation(fig, animate, frames = nframes) 
HTML(ani.to_jshtml())

## 5. Compute simulation error

In [84]:
# MSE:
simulated_T = get_sample(T_tot) 

mse = (np.square(simulated_T - real_T)).mean()

display('The mean squarred error (MSE) for alpha = ' + str(alpha) + ' resulted in '  + str(np.round(mse,3)))

'The mean squarred error (MSE) for alpha = 1.172e-05 resulted in 1.371'

# 6. Please submit your result to the google form for comparsion 

### Link: https://forms.gle/XjQnvJzamnYqvSBh6 